# Topic Modelling on Program Source Code
---
By Kishalay Banerjee, Dan Jones and Sam Harding

In [1]:
import warnings
warnings.filterwarnings('ignore')  # 0y

In [2]:
import pandas
import pickle
import numpy
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [3]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from urllib.request import urlretrieve

In [4]:
numpy.random.seed(0xD00D5) 

## Preparation

There are a number of files which are too large to store on GitHub. These are hosted on our server, and can be downloaded by running the following cell:

In [ ]:
files = [
    'full_lda_model.pickle',
    'full_tf.pickle',
    'full_tf_vectorizer.pickle',
    'full-dataset.csv.gz',
]

base_url = 'https://daniel.wilshirejones.com/private-uUX6IzfsRYLNiti4ZFmgv6U3dFInnq37r5YSQs46iejeB96q0MAy9Ko7hkgo/'
destination_directory = '../data/'

for file in files:
    url = base_url + file
    destination = destination_directory + file
    print("Downloading '{}'' to location '{}'".format(url, destination))
    urlretrieve(url, destination)

## Generating the Dataset

TODO: Import dataset.py and explain how it's used + what it does.

TODO: Add Sam's scrub function in here?

In [5]:
minimal_dataset = pandas.read_csv("../data/dataset.csv.gz", header=None, names=['repo', 'language', 'documents'])
minimal_dataset.head()

,repo,language,documents
0,28457823,javascript,"b""module.exports = {\n plugins: [\n requir..."
1,28457823,javascript,"b""// The path where to mount the REST API app\..."
2,28457823,javascript,"b""import { Observable } from 'rx';\nimport deb..."
3,28457823,javascript,"b""import { Observable } from 'rx';\n// import ..."
4,28457823,javascript,"b""import { Observable } from 'rx';\n\nmodule.e..."


In [6]:
full_dataset = pandas.read_csv("../data/full-dataset.csv.gz", header=None, names=['repo', 'language',  'topics', 'documents'])

# Remove Github 'topics' since we don't use them in this analysis
full_dataset.drop(columns='topics')

full_dataset.head()

,repo,language,topics,documents
0,28457823,javascript,careers;certification;community;curriculum;d3;...,"b""module.exports = {\n plugins: [\n requir..."
1,28457823,javascript,careers;certification;community;curriculum;d3;...,"b""// The path where to mount the REST API app\..."
2,28457823,javascript,careers;certification;community;curriculum;d3;...,"b""import { Observable } from 'rx';\nimport deb..."
3,28457823,javascript,careers;certification;community;curriculum;d3;...,"b""import { Observable } from 'rx';\n// import ..."
4,28457823,javascript,careers;certification;community;curriculum;d3;...,"b""import { Observable } from 'rx';\n\nmodule.e..."


## Topic Modelling on Individual Source Files

Basically done, just need to copy it over. Maybe run on the bigger dataset?

TODO:
  - Copy work from documentation/daniel-jones.ipynb
  - Add visualisation with pyldavis

For our purposes, common words are important and rare words aren't. So we shouldn't use tf-idf as a metric, bag-of-words makes more sense. (TODO: Maybe: "Similarly, filter out words that don't occur very often").


In [6]:
documents = minimal_dataset['documents']

In [7]:
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

We have four programming languages, try to use LDA to determine these four programming languages.

In [8]:
number_of_languages = 4

lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model = lda.fit(tf)

C:\Users\KISHALAY\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [ ]:
with open('../data/minimal_lda_model.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model, f)

In [9]:
with open('../data/minimal_lda_model.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    model = pickle.load(f)

In [10]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.170498  0.068422       1        1  53.661148
3      0.334580 -0.137167       2        1  17.800762
1     -0.215109 -0.203095       3        1  16.993209
2      0.051028  0.271840       4        1  11.544881, topic_info=       Category          Freq           Term         Total  loglift  logprob
term                                                                        
224987  Default  76787.000000           self  76787.000000  30.0000  30.0000
209790  Default  28422.000000          react  28422.000000  29.0000  29.0000
118970  Default  35518.000000           from  35518.000000  28.0000  28.0000
257199  Default  93096.000000           this  93096.000000  27.0000  27.0000
256596  Default  71532.000000            the  71532.000000  26.0000  26.0000
138187  Default  75532.000000             if  75532.000000  25.0000  25.0000
93741   Default  26333.000000        default  26333.000000  24.0000  24.0000
93727   Default  19823.000000            def  19823.000000  23.0000  23.0000
119762  Default  78705.000000       function  78705.000000  22.0000  22.0000
139197  Default  26520.000000         import  26520.000000  21.0000  21.0000
180274  Default  18618.000000        nimport  18618.000000  20.0000  20.0000
178605  Default  13422.000000        nexport  13422.000000  19.0000  19.0000
6203    Default  19257.000000             12  19257.000000  18.0000  18.0000
179191  Default  15713.000000          nfrom  15713.000000  17.0000  17.0000
88624   Default  12159.000000  createsvgicon  12159.000000  16.0000  16.0000
195902  Default  18062.000000           path  18062.000000  15.0000  15.0000
182715  Default  21713.000000           none  21713.000000  14.0000  14.0000
270897  Default  16822.000000           user  16822.000000  13.0000  13.0000
10108   Default  14036.000000             16  14036.000000  12.0000  12.0000
216978  Default  74742.000000         return  74742.000000  11.0000  11.0000
139409  Default  34335.000000             in  34335.000000  10.0000  10.0000
273107  Default  50512.000000            var  50512.000000   9.0000   9.0000
58561   Default  16313.000000             as  16313.000000   8.0000   8.0000
118218  Default   8955.000000       fragment   8955.000000   7.0000   7.0000
4646    Default  16616.000000             10  16616.000000   6.0000   6.0000
281843  Default   7860.000000            xd0   7860.000000   5.0000   5.0000
16221   Default  10334.000000             24  10334.000000   4.0000   4.0000
80378   Default   8276.000000      classname   8276.000000   3.0000   3.0000
281885  Default   7548.000000            xe0   7548.000000   2.0000   2.0000
173909  Default  23687.000000           name  23687.000000   1.0000   1.0000
...         ...           ...            ...           ...      ...      ...
281761   Topic4   1464.391410            xbd   1487.025518   2.1436  -6.2404
281631   Topic4   1663.439660            xb4   1699.407348   2.1375  -6.1129
281809   Topic4   2187.518169            xc3   2248.312349   2.1315  -5.8390
209790   Topic4  26136.460497          react  28422.863801   2.0751  -3.3585
281786   Topic4   1213.602554            xbf   1241.477370   2.1362  -6.4282
80378    Topic4   7546.761037      classname   8276.007692   2.0667  -4.6007
281561   Topic4   1517.149330            xb0   1563.031406   2.1291  -6.2050
281345   Topic4   1061.601815            xa5   1081.515369   2.1403  -6.5620
205348   Topic4   4516.349076      proptypes   4899.945261   2.0774  -5.1141
280778   Topic4   2783.143067            x80   2961.846853   2.0967  -5.5982
281673   Topic4   2447.870929            xb8   2614.659009   2.0930  -5.7266
125631   Topic4   1890.410526        gettext   2012.120308   2.0965  -5.9850
118970   Topic4  25303.791250           from  35518.688459   1.8198  -3.3908
159703   Topic4   3531.937969             m0   4032.249447   2.0265  -5.3600
180274   Topic

In [18]:
number_of_languages = 5

lda2 = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
model2 = lda2.fit(tf)

with open('../data/minimal_lda_model2.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model2, f)

/home/dan/.guix-profile/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [19]:
if not model2:
    with open('../data/minimal_lda_model2.pickle', 'rb') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        model2 = pickle.load(f)

In [20]:
pyLDAvis.sklearn.prepare(model2, tf, tf_vectorizer)

/home/dan/.guix-profile/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.111763  0.166057       1        1  35.756673
0      0.433966 -0.002221       2        1  17.716020
3     -0.100094 -0.054022       3        1  16.634357
4     -0.116689 -0.225309       4        1  15.764461
1     -0.105420  0.115495       5        1  14.128490, topic_info=       Category          Freq           Term         Total  loglift  logprob
term                                                                        
224987  Default  74246.000000           self  74246.000000  30.0000  30.0000
119762  Default  74196.000000       function  74196.000000  29.0000  29.0000
257199  Default  89627.000000           this  89627.000000  28.0000  28.0000
273107  Default  46577.000000            var  46577.000000  27.0000  27.0000
209790  Default  35725.000000          react  35725.000000  26.0000  26.0000
6203    Default  24231.000000             12  24231.000000  25.0000  25.0000
138187  Default  73338.000000             if  73338.000000  24.0000  24.0000
216978  Default  74110.000000         return  74110.000000  23.0000  23.0000
10108   Default  17879.000000             16  17879.000000  22.0000  22.0000
88624   Default  14839.000000  createsvgicon  14839.000000  21.0000  21.0000
256596  Default  69558.000000            the  69558.000000  20.0000  20.0000
93727   Default  19051.000000            def  19051.000000  19.0000  19.0000
16221   Default  13439.000000             24  13439.000000  18.0000  18.0000
270897  Default  17002.000000           user  17002.000000  17.0000  17.0000
263540  Default  33888.000000           true  33888.000000  16.0000  16.0000
25767   Default  11608.000000             48  11608.000000  15.0000  15.0000
5507    Default  12966.000000             11  12966.000000  14.0000  14.0000
20365   Default  11924.000000             32  11924.000000  13.0000  13.0000
139197  Default  26710.000000         import  26710.000000  12.0000  12.0000
8196    Default  11652.000000             14  11652.000000  11.0000  11.0000
15259   Default  10077.000000             21  10077.000000  10.0000  10.0000
7684    Default  11066.000000             13  11066.000000   9.0000   9.0000
179191  Default  15184.000000          nfrom  15184.000000   8.0000   8.0000
4646    Default  18477.000000             10  18477.000000   7.0000   7.0000
118218  Default  10913.000000       fragment  10913.000000   6.0000   6.0000
10837   Default   9684.000000             17   9684.000000   5.0000   5.0000
25152   Default   9606.000000             45   9606.000000   4.0000   4.0000
216449  Default   9694.000000       response   9694.000000   3.0000   3.0000
59010   Default  19018.000000         assert  19018.000000   2.0000   2.0000
59130   Default   7893.000000    assertequal   7893.000000   1.0000   1.0000
...         ...           ...            ...           ...      ...      ...
216978   Topic5  22304.863791         return  74110.619679   0.7562  -3.7189
138187   Topic5  21711.154321             if  73338.722793   0.7397  -3.7459
119762   Topic5  20681.494894       function  74196.078073   0.6795  -3.7945
179216   Topic5   3207.309543      nfunction   6098.926269   1.3143  -5.6583
76176    Topic5   1084.585668             cb   1424.969370   1.6840  -6.7426
234101   Topic5   3401.683926         source   7409.392554   1.1785  -5.5995
217029   Topic5   2018.247337        returns   3523.110365   1.3999  -6.1215
58154    Topic5   2924.411271          array   6036.121075   1.2323  -5.7507
188219   Topic5   2339.381328            obj   4366.244985   1.3330  -5.9739
104097   Topic5   4898.879748           else  14959.810669   0.8406  -5.2347
188243   Topic5   4547.989883         object  13559.271609   0.8646  -5.3091
149943   Topic5   4084.369642            key  11605.641475   0.9127  -5.4166
256596   Topic5  12457.428659            the  69558.594246   0.2371  -4.3014
257199   Topic5  14158.029050        

Try to do this on the full dataset:

In [22]:
number_of_languages = 4
all_documents = full_dataset['documents']

full_tf_vectorizer = CountVectorizer(stop_words=None)
full_tf = full_tf_vectorizer.fit_transform(all_documents)
full_tf_feature_names = full_tf_vectorizer.get_feature_names()

full_lda = LatentDirichletAllocation(n_topics=number_of_languages,  n_jobs=1)
full_model = full_lda.fit(full_tf)

with open('../data/full_lda_model.pickle', 'wb') as f:
    pickle.dump(full_model, f)

with open('../data/full_tf.pickle', 'wb') as f:
    pickle.dump(full_tf, f)
    
with open('../data/full_tf_vectorizer.pickle', 'wb') as f:
    pickle.dump(full_tf_vectorizer, f)

/home/dan/.guix-profile/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [7]:
with open('../data/full_lda_model.pickle', 'rb') as f:
    full_model = pickle.load(f)

with open('../data/full_tf.pickle', 'rb') as f:
    full_tf = pickle.load(f)
    
with open('../data/full_tf_vectorizer.pickle', 'rb') as f:
    full_tf_vectorizer = pickle.load(f)

Visualize our new model. Note that the following code cell requires more than 8 GB of RAM to run.

In [ ]:
pyLDAvis.sklearn.prepare(full_model, full_tf, full_tf_vectorizer)

Reference: https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb

## Topic Modelling on Programming Language Mixtures
Keypoint: topics are programming languages, file with mixture of programming languages, identify which is which.

Applicability to cyber-security: identifying malware embedded within normal programs (shellcode).

### Combining Source Files per Repository

TODO:
  - Copy work Sam's
  
  
### LDA Model

TODO:
  - Write model
  - Add visualisation with pyldavis
 

## Identifying Program Subjects and Themes (DAN)

Hypothesis: Using tf-idf rather than bag-of-words as an input to LDA will prioritise rare words. In the case of source code, this means programming language keywords (an identifying feature of programming languages) are deprioritised, and so a more human idea of topics may emerge. 

We can use repo-list.json and the repo-ids to map the github topics/tags to each repo. Might be a small/easy task to compare against the programming langauge identification.

In [29]:
documents = minimal_dataset['documents']

In [30]:
tf_vectorizer = TfidfVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

We have four programming languages, try to use LDA to determine these four programming languages.

In [31]:
number_of_themes = 3

lda = LatentDirichletAllocation(n_topics=number_of_themes,  n_jobs=1)
model = lda.fit(tf)

/home/dan/.guix-profile/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [32]:
with open('../data/tfidf_lda_model.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model, f)

In [33]:
with open('../data/tfidf_lda_model.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    model = pickle.load(f)

In [34]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

/home/dan/.guix-profile/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.063454 -0.019868       1        1  50.348580
1      0.071515 -0.013831       2        1  30.512626
0     -0.008060  0.033699       3        1  19.138794, topic_info=       Category        Freq           Term       Total  loglift  logprob
term                                                                    
88624   Default  192.000000  createsvgicon  192.000000  30.0000  30.0000
209790  Default  230.000000          react  230.000000  29.0000  29.0000
118218  Default  127.000000       fragment  127.000000  28.0000  28.0000
195902  Default  144.000000           path  144.000000  27.0000  27.0000
25152   Default   87.000000             45   87.000000  26.0000  26.0000
224987  Default  140.000000           self  140.000000  25.0000  25.0000
28637   Default   63.000000             55   63.000000  24.0000  24.0000
159703  Default   62.000000             m0   62.000000  23.0000  23.0000
113922  Default   62.000000           fill   62.000000  22.0000  22.0000
2713    Default   54.000000   0h24v24h0v0z   54.000000  21.0000  21.0000
179191  Default   81.000000          nfrom   81.000000  20.0000  20.0000
271947  Default   62.000000          utils   62.000000  19.0000  19.0000
6203    Default   51.000000             12   51.000000  18.0000  18.0000
118970  Default  134.000000           from  134.000000  17.0000  17.0000
24129   Default   41.000000             41   41.000000  16.0000  16.0000
32410   Default   39.000000             67   39.000000  15.0000  15.0000
21298   Default   39.000000             34   39.000000  14.0000  14.0000
22630   Default   39.000000             39   39.000000  13.0000  13.0000
256596  Default  122.000000            the  122.000000  12.0000  12.0000
138187  Default   88.000000             if   88.000000  11.0000  11.0000
29494   Default   36.000000             59   36.000000  10.0000  10.0000
5507    Default   48.000000             11   48.000000   9.0000   9.0000
85308   Default   48.000000          const   48.000000   8.0000   8.0000
167322  Default   61.000000     migrations   61.000000   7.0000   7.0000
41134   Default   35.000000             99   35.000000   6.0000   6.0000
182715  Default   75.000000           none   75.000000   5.0000   5.0000
25767   Default   33.000000             48   33.000000   4.0000   4.0000
7684    Default   35.000000             13   35.000000   3.0000   3.0000
10837   Default   31.000000             17   31.000000   2.0000   2.0000
92062   Default   57.000000           data   57.000000   1.0000   1.0000
...         ...         ...            ...         ...      ...      ...
145278   Topic3    5.541038     isrequired    5.955611   1.5813  -8.0900
280154   Topic3    9.544478        wrapper   10.592366   1.5493  -7.5462
169778   Topic3    9.338290        modules   10.400661   1.5457  -7.5680
205348   Topic3   22.228383      proptypes   26.125031   1.4919  -6.7008
99391    Topic3   11.588912           docs   13.401489   1.5081  -7.3521
68795    Topic3    9.211198      bootstrap   10.547685   1.5180  -7.5817
267055   Topic3   17.413034             ui   21.262382   1.4537  -6.9449
235758   Topic3   13.549351            src   16.329754   1.4668  -7.1958
218923   Topic3   12.140906           root   14.867368   1.4509  -7.3056
215527   Topic3   15.877889        require   21.325279   1.3585  -7.0372
162590   Topic3    9.806341       markdown   12.335538   1.4240  -7.5191
256735   Topic3   14.133347          theme   20.221678   1.2952  -7.1536
175955   Topic3   15.498111         nconst   22.718727   1.2710  -7.0614
85308    Topic3   25.938984          const   48.539039   1.0268  -6.5464
72266    Topic3   14.362309         button   25.619311   1.0747  -7.1376
86632    Topic3   12.746794           core   21.742592   1.1195  -7.2569
84071    Topic3    9.744030     components   15.044200   1.2191  -7.5255
109172   Topic3   12.835990         e

This still prioritises programming language keywords. One approach to solving this problem is to consider all keywords as "stopwords". First, gather a list of R, Python and Javascript keywords:

In [11]:
import keyword

python_keywords = keyword.kwlist
python_keywords

['False',
 'None',
 'True',
 'and',
 'as',
 'assert',
 'break',
 'class',
 'continue',
 'def',
 'del',
 'elif',
 'else',
 'except',
 'finally',
 'for',
 'from',
 'global',
 'if',
 'import',
 'in',
 'is',
 'lambda',
 'nonlocal',
 'not',
 'or',
 'pass',
 'raise',
 'return',
 'try',
 'while',
 'with',
 'yield']

R reserved words (sourced from the manual: https://stat.ethz.ch/R-manual/R-devel/library/base/html/Reserved.html)

In [12]:
r_keywords = [
    "if", 
    "else", 
    "repeat",
    "while",
    "function", 
    "for",
    "in",
    "next",
    "break",
    "TRUE",
    "FALSE",
    "NULL", 
    "Inf", 
    "NaN",
    "NA",
    "NA_integer_",
    "NA_real_",
    "NA_complex_",
    "NA_character_", 
]

Javascript keywords and reserved words (source: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Lexical_grammar#Keywords)

In [13]:
javascript_keywords = [
    "break",
    "case",
    "catch",
    "class",
    "const",
    "continue",
    "debugger",
    "default",
    "delete",
    "do",
    "else",
    "export",
    "extends",
    "finally",
    "for",
    "function",
    "if",
    "import",
    "in",
    "instanceof",
    "new",
    "return",
    "super",
    "switch",
    "this",
    "throw",
    "try",
    "typeof",
    "var",
    "void",
    "while",
    "with",
    "yield",
]

In [38]:
documents = minimal_dataset['documents']

In [39]:
tf_vectorizer = TfidfVectorizer(stop_words=javascript_keywords+python_keywords+r_keywords)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

/home/dan/.guix-profile/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['false', 'inf', 'na', 'na_character_', 'na_complex_', 'na_integer_', 'na_real_', 'nan', 'none', 'null', 'true'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


We have four programming languages, try to use LDA to determine these four programming languages.

In [40]:
number_of_themes = 3

lda = LatentDirichletAllocation(n_topics=number_of_themes,  n_jobs=1)
model = lda.fit(tf)

/home/dan/.guix-profile/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [41]:
with open('../data/tfidf_lda_model_ignore_keywords.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model, f)

In [42]:
with open('../data/tfidf_lda_model_ignore_keywords.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    model = pickle.load(f)

In [43]:
pyLDAvis.sklearn.prepare(model, tf, tf_vectorizer)

/home/dan/.guix-profile/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.047957  0.027118       1        1  40.458879
0      0.073447  0.006158       2        1  32.147917
2     -0.025490 -0.033276       3        1  27.393204, topic_info=       Category        Freq           Term       Total  loglift  logprob
term                                                                    
88613   Default  195.000000  createsvgicon  195.000000  30.0000  30.0000
209750  Default  243.000000          react  243.000000  29.0000  29.0000
118194  Default  129.000000       fragment  129.000000  28.0000  28.0000
195863  Default  146.000000           path  146.000000  27.0000  27.0000
224945  Default  120.000000           self  120.000000  26.0000  26.0000
25152   Default   88.000000             45   88.000000  25.0000  25.0000
28637   Default   63.000000             55   63.000000  24.0000  24.0000
159670  Default   63.000000             m0   63.000000  23.0000  23.0000
113900  Default   63.000000           fill   63.000000  22.0000  22.0000
2713    Default   55.000000   0h24v24h0v0z   55.000000  21.0000  21.0000
271899  Default   63.000000          utils   63.000000  20.0000  20.0000
24129   Default   42.000000             41   42.000000  19.0000  19.0000
6203    Default   51.000000             12   51.000000  18.0000  18.0000
167289  Default   52.000000     migrations   52.000000  17.0000  17.0000
32410   Default   40.000000             67   40.000000  16.0000  16.0000
21298   Default   39.000000             34   39.000000  15.0000  15.0000
22630   Default   39.000000             39   39.000000  14.0000  14.0000
179156  Default   67.000000          nfrom   67.000000  13.0000  13.0000
256552  Default  106.000000            the  106.000000  12.0000  12.0000
29494   Default   36.000000             59   36.000000  11.0000  11.0000
205309  Default   31.000000      proptypes   31.000000  10.0000  10.0000
5507    Default   48.000000             11   48.000000   9.0000   9.0000
41134   Default   35.000000             99   35.000000   8.0000   8.0000
98809   Default   42.000000         django   42.000000   7.0000   7.0000
25767   Default   34.000000             48   34.000000   6.0000   6.0000
7684    Default   35.000000             13   35.000000   5.0000   5.0000
10837   Default   31.000000             17   31.000000   4.0000   4.0000
270849  Default   39.000000           user   39.000000   3.0000   3.0000
35702   Default   29.000000             79   29.000000   2.0000   2.0000
182680  Default   72.000000           none   72.000000   1.0000   1.0000
...         ...         ...            ...         ...      ...      ...
128982   Topic3    9.623524           grid    9.955150   1.2610  -7.8619
209856   Topic3    6.777216       reactdom    7.021972   1.2594  -8.2125
267007   Topic3   24.575764             ui   25.820444   1.2455  -6.9243
280102   Topic3   13.112242        wrapper   13.696403   1.2513  -7.5526
108668   Topic3   15.204472         expect   16.143106   1.2350  -7.4045
82662    Topic3   16.307323          color   17.412072   1.2293  -7.3345
96586    Topic3    9.467231             df   10.015851   1.2385  -7.8783
218881   Topic3   16.519929           root   18.421415   1.1859  -7.3215
72260    Topic3   23.324213         button   27.409646   1.1335  -6.9766
256691   Topic3   19.460698          theme   23.559825   1.1037  -7.1577
205279   Topic3   25.065865          props   33.367129   1.0088  -6.9046
98657    Topic3   25.667336            div   34.441207   1.0008  -6.8809
234059   Topic3   17.345423         source   21.657322   1.0729  -7.2728
278299   Topic3   11.438914          width   13.128324   1.1571  -7.6891
80369    Topic3   22.542996      classname   30.049873   1.0074  -7.0107
239589   Topic3   11.517111          style   13.386386   1.1445  -7.6823
86621    Topic3   17.142016           core   24.374993   0.9429  -7.2846
79084    Topic3   10.652365       chi

## Measuring the Efficacy of Topic Models (WRITEUP: WHO?)

Main question: How do we evaluate how well a topic (from LDA for example) represents a meaningful "topic" or theme?

TODO: do some research on this??? There must be some papers etc that try to formalise this that we can borrow ideas from?

Paper dump:
  - Looks like a good summary paper: http://www.aclweb.org/anthology/E14-4005 Find more papers from this ones references?
    - " KL-divergence (Li and McCallum, 2006; Wang et al., 2009; Newman et al., 2009), cosine measure (He et al., 2009; Ramage et al., 2009) and the average Log Odds Ratio (Chaney and Blei, 2012). "
    - "Kim and Oh (2011) also applied  the  cosine  measure  and  KL-Divergence which were compared with four other measures: Jaccard’s Coefficient, Kendall’s τ coefficient, Discount  Cumulative  Gain  and  Jensen  Shannon  Divergence (JSD)."
  - Cool name haven't read it: http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf

### Reading Tea Leaves Paper
http://papers.nips.cc/paper/3700-reading-tea-leaves-how-humans-interpret-topic-models.pdf

### Word Overlap

I think this is used as a baseline measure in the summary paper above (http://www.aclweb.org/anthology/E14-4005). Should be a quick implementation so worth a try.

### Jaccard Index

From the papers above this seems to have been used relatively often for _linking machine-generated topics to human topics_ and so maybe this is a good application for it. Apparently explored here "https://link.springer.com/chapter/10.1007/978-3-642-19437-5_13" but I haven't read it.


In [16]:
def jaccard_index(a, b):
    a = set(a)
    b = set(b)
    return len(a & b) / len(a | b)

Consider the group of language keywords as the best possible topic for each language. Compare each of our machine generated topics with each of our ideal topics by computing their Jaccard Index:

In [23]:
topic1_keywords = [

"this",
"function",
"if",
"the",
"return",
"var",
"to",
"for",
"is",
"true",
"in",
"of",
"value",
"data",
"null",
"length",
"and",
"else",
"false",
"name",
"type",
"new",
"const",
"it",
"assert",
"object",
"be",
"options",
"key",
"that"]

In [31]:
topic3_keywords = [
"self",
"def",
"import",
"from",
"none",
"user",
"in",
"ndef",
"response",
"name",
"assert",
"equal",
"id",
"not",
"str",
"models",
"true",
"request",
"get",
"email",
"assert",
"realm",
"data",
"message",
"result",
"for",
"nclass",
"dict",
"django",
"thread",
"url"]

In [35]:
topic4_keywords = [
"react",
"from",
"default",
"nimport",
"nexport",
"createsvgicon",
"path",
"import",
"fragment",
"xd0",
"xe0",
"classname",
"as",
"props",
"utils",
"none",
"div",
"proptypes",
"createelement",
"theme",
"xe1",
"material",
"xd1",
"button",
"classes",
"m0",
"fill",
"ui",
"xe2",
"0h24v24h0v0z"]

In [41]:
print(jaccard_index(javascript_keywords, topic1_keywords),
jaccard_index(javascript_keywords, topic3_keywords),
jaccard_index(javascript_keywords, topic4_keywords))

0.18867924528301888 0.05 0.03278688524590164


In [42]:
print(jaccard_index(r_keywords, topic1_keywords),
jaccard_index(r_keywords, topic3_keywords),
jaccard_index(r_keywords, topic4_keywords))

0.11363636363636363 0.0425531914893617 0.0


In [43]:
print(jaccard_index(python_keywords, topic1_keywords),
jaccard_index(python_keywords, topic3_keywords),
jaccard_index(python_keywords, topic4_keywords))

0.14545454545454545 0.125 0.05


In [29]:
len(python_keywords)

33

In [55]:
with open('../data/minimal_lda_model.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    model = pickle.load(f)
    
tf_vectorizer = CountVectorizer(stop_words=None)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

/home/dan/.guix-profile/lib/python3.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [54]:
len(tf_feature_names)

284872

In [57]:
list(enumerate(model.components_))

[(0, array([238.92845592, 148.74385946,  66.36479644, ...,   0.25075697,
           0.25075697,   0.25075697])),
 (1, array([2.99244369e+02, 2.57844111e-01, 3.16097064e-01, ...,
         2.50141731e-01, 2.50141731e-01, 2.50141731e-01])),
 (2, array([37.52458607,  0.26637312,  0.29111693, ...,  0.2500025 ,
          0.2500025 ,  0.2500025 ])),
 (3, array([0.26618775, 6.41071354, 0.25905608, ..., 1.38201933, 1.38201933,
         1.38201933]))]

### Kendall’s τ Coefficient

Measures the association between two ranked lists. Source: Computational Linguistics and Intelligent Text Processing book.

### Evaluating Topic Models
TODO: better title needed

Idea:
  - save the actual % of each program langauge per repo
  - Then try to use LDA model to tell us "I believe repo <x> is 10% Topic 1, 20% Topic 2 etc". 
  - Use analysis from above two sections to create a "most likely mapping from lda topic to programming language".
  - rate our models

Here we can do cross-validation etc.